In [206]:
from torch.autograd.functional import hessian
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

In [207]:
class LinearRegression(nn.Module):
    def __init__(self, input_dim):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return self.linear(x)

In [208]:
def neg_log_likelihood(y_pred, y, log_sigma):
    sigma = torch.exp(log_sigma)
    return torch.sum((y_pred - y)**2 / (2 * sigma**2) + log_sigma + 0.5 * torch.log(2 * torch.tensor(np.pi)))

In [209]:
# Generate synthetic data
n = 1000
X = np.random.rand(n, 1)
Y =  3 * X  + 1.0 * np.random.randn(n, 1)

# Convert to PyTorch tensors
X_torch = torch.tensor(X, dtype=torch.float32)
Y_torch = torch.tensor(Y, dtype=torch.float32)

In [210]:
input_dim = 1
learning_rate = 0.01
num_epochs = 3000

model = LinearRegression(input_dim)
log_sigma = torch.tensor(0.0, requires_grad=True)
optimizer = optim.Adam([{'params': model.parameters()}, {'params': [log_sigma]}], lr=learning_rate)

In [211]:
for epoch in range(num_epochs):
    optimizer.zero_grad()

    # Forward pass
    Y_pred = model(X_torch)

    # Compute the negative log-likelihood
    loss = neg_log_likelihood(Y_pred, Y_torch, log_sigma)

    # Backward pass
    loss.backward()

    # Update the parameters
    optimizer.step()

    if (epoch+1) % 500 == 0:
        print(f'Epoch: {epoch+1}, Loss: {loss.item()}')


Epoch: 500, Loss: 1500.0025634765625
Epoch: 1000, Loss: 1457.915771484375
Epoch: 1500, Loss: 1456.65185546875
Epoch: 2000, Loss: 1456.647216796875
Epoch: 2500, Loss: 1456.6473388671875
Epoch: 3000, Loss: 1456.647216796875


In [212]:
w, b = model.linear.weight.detach().numpy(), model.linear.bias.detach().numpy()
estimated_sigma = torch.exp(log_sigma).detach().numpy()
print(f'Estimated parameters: w = {w}, b = {b}, sigma = {estimated_sigma}')

Estimated parameters: w = [[2.813105]], b = [0.13518801], sigma = 1.0384286642074585


In [213]:
from torch.autograd import grad

In [221]:
w = torch.tensor([[3.0]], dtype=torch.float32)
b =  torch.tensor([0.0], dtype=torch.float32)
w.requires_grad = True
b.requires_grad = True

In [230]:
def neg_ll_hessian(w, b, log_sigma):
    #new_mdl = LinearRegression(input_dim)
    #new_mdl.linear.weight.data = w
    #new_mdl.linear.bias.data = b
    #Y_pred = new_mdl(X_torch)
    Y_pred = X_torch * w + b
    return neg_log_likelihood(Y_pred, Y_torch, log_sigma)

In [231]:
neg_ll_hessian(w, b, log_sigma)

tensor(1458.6542, grad_fn=<SumBackward0>)

In [232]:
hessian(neg_ll_hessian, params)

((tensor([[[[223.8602]]]]), tensor([[[330.2464]]]), tensor([[-1704.2228]])),
 (tensor([[[330.2464]]]), tensor([[635.8584]]), tensor([-3663.2722])),
 (tensor([[-1704.2230]]), tensor([-3663.2722]), tensor(12299.5908)))